# Derivation of Kalman Filter update and predict equations

The kinematic assumptions of this filter is that the body's acceleration is only due to gravity.

The filter states are:

| State Var | Description            | Units   |
|:-----------|:------------------------|:---------:|
| $q_0$      | Quaternion    |  |
| $q_1$      | Quaternion   |  |
| $q_2$      | Quaternion     |  |
| $q_3$      | Quaternion     |  |
| $\omega_x$     | Body axis rotation rate | rad/s |
| $\omega_y$     | Body axis rotation rate | rad/s |
| $\omega_z$     | Body axis rotation rate | rad/s |
| $\omega_{bx}$     | Gyro bias | rad/s |
| $\omega_{by}$     | Gyro bias | rad/s |
| $\omega_{bz}$     | Gyro bias | rad/s |
| $a_{bx}$     | Accel bias | m/s^2 |
| $a_{by}$     | Accel bias | m/s^2 |
| $a_{bz}$     | Accel bias | m/s^2 |

Kalman Filter parameters:
    
| Parameter | Description |    
| :--- | :--- |
| $x$ | State vector ($n_{states} \times 1$ ) |
| $P$ | Covariance of states ($n_{states} \times n_{states}$) |
| $Q$ | Covariance of the prediction step. Can be thought of as how incorrect is the model used in the prediction step. ($n_{states} \times n_{states}$) |
| $F$ | For a standard Kalman filter, this is the matrix that when multiplied by $x$ generates the model prediction. ($n_{states} \times n_{states}$) |
| $R$ | Covariance of sensors ($n_{sensors} \times n_{sensors}$) |
| $H$ | Maps $x$ to sensor space. ($n_{sensors} \times n_{states}$) |

Prediction:
$$
\boldsymbol{x_{k | k-1}} = f\left( x_{k-1 | k-1} \right) 
$$
$$ P = F P F^T + Q $$

Update:
$$ 
\begin{align}
\boldsymbol{y_k} &= z_k - h\left( x_{k | k-1} \right) \\
S &= H P H^T + R \\
K &= P H^T S^{-1} \\
\boldsymbol{x_{k|k}} &= x_{k | k-1} + K y \\
P &= (I - K H) P \\ 
\end{align}
$$ 

$F$ and $H$ are the Jacobian matricies corresponding to the functions $f()$ and $h()$ respectively.

## Kalman Filter Prediction Equations

In [1]:
from sympy import symbols, sqrt, diff, simplify
import quaternion
from quaternion import Quaternion
import numpy as np

quaternion.sqrt = sqrt

q0, q1, q2, q3 = symbols(['q0', 'q1', 'q2', 'q3'])
wx, wy, wz = symbols(['wx', 'wy', 'wz'])
wbx, wby, wbz = symbols(['wbx', 'wby', 'wbz'])
abx, aby, abz = symbols(['abx', 'aby', 'abz'])

# Magnetometer sensor output
mx, my, mz = symbols(['mx', 'my', 'mz'])
dt = symbols('dt')
states = [q0, q1, q2, q3, wx, wy, wz, wbx, wby, wbz, abx, aby, abz]
# Predict

q = Quaternion(q0, q1, q2, q3) * Quaternion(1.0, .5*wx*dt, .5*wy*dt, .5*wz*dt)
#q.normalize()

nextstate = [q[0], q[1], q[2], q[3], wx, wy, wz, wbx, wby, wbz, abx, aby, abz]
for n in range(7):
    print(f"newq[{n}] = {nextstate[n]}")

F = np.zeros((13, 13), dtype=object)
for n in range(13):
    for m in range(13):
        tmp = (diff(nextstate[n], states[m]))
        F[n, m] = tmp
        if tmp != 0:
            print(f"F[{n}, {m}] = {tmp}")


newq[0] = -0.5*dt*q1*wx - 0.5*dt*q2*wy - 0.5*dt*q3*wz + 1.0*q0
newq[1] = 0.5*dt*q0*wx + 0.5*dt*q2*wz - 0.5*dt*q3*wy + 1.0*q1
newq[2] = 0.5*dt*q0*wy - 0.5*dt*q1*wz + 0.5*dt*q3*wx + 1.0*q2
newq[3] = 0.5*dt*q0*wz + 0.5*dt*q1*wy - 0.5*dt*q2*wx + 1.0*q3
newq[4] = wx
newq[5] = wy
newq[6] = wz
F[0, 0] = 1.00000000000000
F[0, 1] = -0.5*dt*wx
F[0, 2] = -0.5*dt*wy
F[0, 3] = -0.5*dt*wz
F[0, 4] = -0.5*dt*q1
F[0, 5] = -0.5*dt*q2
F[0, 6] = -0.5*dt*q3
F[1, 0] = 0.5*dt*wx
F[1, 1] = 1.00000000000000
F[1, 2] = 0.5*dt*wz
F[1, 3] = -0.5*dt*wy
F[1, 4] = 0.5*dt*q0
F[1, 5] = -0.5*dt*q3
F[1, 6] = 0.5*dt*q2
F[2, 0] = 0.5*dt*wy
F[2, 1] = -0.5*dt*wz
F[2, 2] = 1.00000000000000
F[2, 3] = 0.5*dt*wx
F[2, 4] = 0.5*dt*q3
F[2, 5] = 0.5*dt*q0
F[2, 6] = -0.5*dt*q1
F[3, 0] = 0.5*dt*wz
F[3, 1] = 0.5*dt*wy
F[3, 2] = -0.5*dt*wx
F[3, 3] = 1.00000000000000
F[3, 4] = -0.5*dt*q2
F[3, 5] = 0.5*dt*q1
F[3, 6] = 0.5*dt*q0
F[4, 4] = 1
F[5, 5] = 1
F[6, 6] = 1
F[7, 7] = 1
F[8, 8] = 1
F[9, 9] = 1
F[10, 10] = 1
F[11, 11] = 1
F[12, 12] = 

## Accelerometer update

In [2]:
q = Quaternion(q0, q1, q2, q3)
g = np.array([0,0,9.8], dtype=float)
accel = (q.inv() * Quaternion.from_vec(g) * q).as_ndarray()[1:] + np.array([abx, aby, abz])
H = np.zeros((3, 13), dtype=object)
for n in range(3):
    for m in range(13):
        tmp = diff(accel[n], states[m])
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 0] = 39.2*q0**2*q2/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 39.2*q0*q1*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 19.6*q2/(q0**2 + q1**2 + q2**2 + q3**2)
H[0, 1] = 39.2*q0*q1*q2/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 39.2*q1**2*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 19.6*q3/(q0**2 + q1**2 + q2**2 + q3**2)
H[0, 2] = 39.2*q0*q2**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 19.6*q0/(q0**2 + q1**2 + q2**2 + q3**2) - 39.2*q1*q2*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2
H[0, 3] = 39.2*q0*q2*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 39.2*q1*q3**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 19.6*q1/(q0**2 + q1**2 + q2**2 + q3**2)
H[0, 10] = 1
H[1, 0] = -39.2*q0**2*q1/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 39.2*q0*q2*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 19.6*q1/(q0**2 + q1**2 + q2**2 + q3**2)
H[1, 1] = -39.2*q0*q1**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 19.6*q0/(q0**2 + q1**2 + q2**2 + q3**2) - 39.2*q1*q2*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2
H[1, 2] = -39.2*q0*q1*q2/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 39.2*q2**2*

## Gyro update

In [3]:
gyros = np.array([wx,wy,wz]) + np.array([wbx, wby, wbz])
H = np.zeros((3, 13), dtype=object)
for n in range(3):
    for m in range(13):
        tmp = diff(gyros[n], states[m])
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 4] = 1
H[0, 7] = 1
H[1, 5] = 1
H[1, 8] = 1
H[2, 6] = 1
H[2, 9] = 1


## Magnetometer Update


Only consider the normalized vector projected onto the global xy plane

1.  Rotate the magnetometer measurement to global coordinates
2.  Only look at component in xy plane and normalize.
3.  Rotate this back to body coordinates -- this is the "measurement

Kalman eqn:
$$y = z - H*x$$
innovation = measurement - (func translate state to measurement)

In [4]:
q = Quaternion(q0, q1, q2, q3)
mag_inertial = (q * Quaternion.from_vec(np.array([mx, my, mz])) * q.inv()).as_ndarray()[1:]
mag_inertial[2] = 0
mag_inertial /= sqrt(mag_inertial[0]**2 + mag_inertial[1]**2)

mag_body = (q.inv() * Quaternion.from_vec(mag_inertial) * q).as_ndarray()[1:]

# Only portion that we need to determin Jacobian is transvering "north vector"
# to body coordinates

north_body = (q.inv() * Quaternion.from_vec(np.array([1.0, 0, 0])) * q).as_ndarray()[1:]
H = np.zeros((3, 13), dtype=object)
for n in range(3):
    for m in range(13):
        tmp = diff(north_body[n], states[m])
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 0] = -2.0*q0**3/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 2.0*q0*q1**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q0*q2**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q0*q3**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q0/(q0**2 + q1**2 + q2**2 + q3**2)
H[0, 1] = -2.0*q0**2*q1/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 2.0*q1**3/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q1*q2**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q1*q3**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q1/(q0**2 + q1**2 + q2**2 + q3**2)
H[0, 2] = -2.0*q0**2*q2/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 2.0*q1**2*q2/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q2**3/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q2*q3**2/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 2.0*q2/(q0**2 + q1**2 + q2**2 + q3**2)
H[0, 3] = -2.0*q0**2*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 2.0*q1**2*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q2**2*q3/(q0**2 + q1**2 + q2**2 + q3**2)**2 + 2.0*q3**3/(q0**2 + q1**2 + q2**2 + q3**2)**2 - 2.0*q3/(q0**2 + q1**2 + q2**2 + q3**2)
H[1, 0] = 4.